In [25]:
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive is mounted at: /content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive is mounted at: /content/drive


In [26]:
# Problem 1
import csv

def hash_join_R1_R2(R1_rows, R2_rows):
    # Hash R2 on attribute B
    bucket_by_B = {}

    for (b_value, c_value) in R2_rows:
        if b_value not in bucket_by_B:
            bucket_by_B[b_value] = []
        bucket_by_B[b_value].append((b_value, c_value))

    # Probe with R1 on attribute B
    result_rows = []

    for (a_value, b_value) in R1_rows:
        if b_value in bucket_by_B:
            for (_, c_value) in bucket_by_B[b_value]:
                result_rows.append((a_value, b_value, c_value))

    return result_rows

def load_csv(filename):
    rows = []
    with open(filename, "r") as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            rows.append((int(row[0]), int(row[1])))
    return rows


# Read R1 and R2
R1_rows = load_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/Problem_1_R1.csv")
R2_rows = load_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/Problem_1_R2.csv")


# Run the join
joined_rows = hash_join_R1_R2(R1_rows, R2_rows)

for row in joined_rows:
    print(row)

(1, 5, 42)
(1, 5, 87)
(2, 7, 13)
(2, 7, 99)
(3, 5, 42)
(3, 5, 87)
(4, 9, 76)
(6, 3, 58)
(8, 2, 144)
(9, 7, 13)
(9, 7, 99)
(10, 4, 31)
(11, 5, 42)
(11, 5, 87)
(13, 8, 222)


In [27]:
# Problem 2: Line Join using a simplified Yannakakis algorithm

from collections import defaultdict
import csv


def semijoin_reduce(left_rows, right_rows, left_attr, right_attr):
    support_values = {row[right_attr] for row in right_rows}
    return [row for row in left_rows if row[left_attr] in support_values]


def yannakakis_line_join(relations):
    """
    q(A1,...,Ak+1) :- R1(A1,A2), R2(A2,A3), ..., Rk(Ak,Ak+1)

    Input:
        relations = [R1, R2, ..., Rk]
        Each Ri is a list of (Ai, Ai+1)

    Output:
        Full join result producing (A1,...,Ak+1)
    """

    k = len(relations)
    if k == 0:
        return []

    tables = [list(R) for R in relations]

    # 1. Bottom-Up Semijoin Pass
    for i in range(k - 1, 0, -1):
        # Ri joins with Ri+1 on Ai+1
        tables[i - 1] = semijoin_reduce(
            left_rows=tables[i - 1],
            right_rows=tables[i],
            left_attr=1,    # Ai+1 in Ri
            right_attr=0    # Ai+1 in Ri+1
        )

    # 2. Top-Down Semijoin Pass (Left → Right)
    for i in range(k - 1):
        tables[i + 1] = semijoin_reduce(
            left_rows=tables[i + 1],
            right_rows=tables[i],
            left_attr=0,    # Ai+1 in Ri+1
            right_attr=1    # Ai+1 in Ri
        )

    # 3. Final Join Enumeration
    result = [(a1, a2) for (a1, a2) in tables[0]]

    for i in range(1, k):
        next_table = tables[i]

        # Hash for fast join
        index = defaultdict(list)
        for (ai, ai_next) in next_table:
            index[ai].append(ai_next)

        new_result = []
        for tup in result:
            join_key = tup[-1]
            if join_key in index:
                for val in index[join_key]:
                    new_result.append(tup + (val,))

        result = new_result

    return result

# CSV Loader
def load_csv(filename):
    rows = []
    with open(filename, "r") as f:
        reader = csv.reader(f)
        next(reader)  # skip header
        for row in reader:
            rows.append((int(row[0]), int(row[1])))
    return rows


# Load Problem 2 Relations
R1_rows = load_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/Problem_2_R1.csv")
R2_rows = load_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/Problem_2_R2.csv")
R3_rows = load_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/Problem_2_R3.csv")

relations = [R1_rows, R2_rows, R3_rows]

print("Output of Problem 2:")
result_rows = yannakakis_line_join(relations)

for row in result_rows:
    print(row)


Output of Problem 2:
(10, 20, 200, 900)
(11, 22, 220, 950)
(13, 25, 250, 999)
(15, 30, 300, 888)


In [28]:
# Problem 3: line join
# q(A1,...,A{k+1}) :- R1(A1,A2), R2(A2,A3), ..., Rk(Ak,Ak+1)

from collections import defaultdict
import csv

def hash_join_step(left_rows, right_rows):
    """
    Performs one join step in a left-deep plan.

    left_rows: tuples (A1, ..., Ai)
    right_rows: tuples (Ai, Ai+1)

    Output: tuples (A1, ..., Ai, Ai+1)
    """

    # Build hash table on right_rows over the attribute Ai
    join_index = defaultdict(list)
    for (a_value, next_value) in right_rows:
        join_index[a_value].append(next_value)

    # Perform the join
    new_rows = []
    for tup in left_rows:
        join_key = tup[-1]         # last attribute Ai
        if join_key in join_index:
            for val in join_index[join_key]:
                new_rows.append(tup + (val,))   # extend tuple

    return new_rows


def simple_line_join(relations):
    num_relations = len(relations)
    if num_relations == 0:
        return []

    # Start with R1: tuples of (A1, A2)
    result_rows = [(a, b) for (a, b) in relations[0]]

    # Join step-by-step with R2, R3, ..., Rk
    for i in range(1, num_relations):
        next_relation = relations[i]
        result_rows = hash_join_step(result_rows, next_relation)

    return result_rows


def load_csv(filename):
    rows = []
    with open(filename, "r") as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            rows.append((int(row[0]), int(row[1])))
    return rows

R1_rows = load_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/Problem_3_R1.csv")
R2_rows = load_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/Problem_3_R2.csv")
R3_rows = load_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/Problem_3_R3.csv")

# Example with k = 3 (three relations)
relations = [R1_rows, R2_rows, R3_rows]

print("Output for Problem 3:")
final_output = simple_line_join(relations)

for row in final_output:
    print(row)


Output for Problem 3:
(10, 100, 200, 3000)
(11, 110, 210, 3100)
(12, 120, 220, 3200)


In [29]:
# Problem 4
import random
import time
from collections import defaultdict, Counter

# Dataset generation
random.seed(2025)

def make_common_values(num_common=20, max_val=100):
    # choose common join values from 1..100 so R3=(l,l) can match
    return random.sample(range(1, max_val + 1), num_common)

def build_relations(num_tuples=100, commons=None):
    # R1: (i, x) where x in commons
    # R2: (y, j) where y in commons
    # R3: (l, l) for l = 1..num_tuples
    if commons is None:
        commons = make_common_values()
    R1 = [(i, random.choice(commons)) for i in range(1, num_tuples + 1)]
    R2 = [(random.choice(commons), j) for j in range(1, num_tuples + 1)]
    R3 = [(l, l) for l in range(1, num_tuples + 1)]
    return R1, R2, R3

# create dataset
common_vals = make_common_values(20, 100)
R1, R2, R3 = build_relations(100, common_vals)

# Simplified Yannakakis
class SimpleYannakakis:
    def __init__(self, relations):
        # relations is a list of 2-tuples lists representing R1(A1,A2), R2(A2,A3), ...
        # copy inputs to avoid mutating caller's lists
        self.rels = [list(r) for r in relations]

    @staticmethod
    def semijoin(left, right, left_join_idx=1, right_join_idx=0):

        support = {t[right_join_idx] for t in right}
        return [t for t in left if t[left_join_idx] in support]

    def reduce_bottom_up(self):
        # bottom-up: for i = k-1 downto 1, reduce R_i using R_{i+1}
        for i in range(len(self.rels) - 2, -1, -1):
            self.rels[i] = SimpleYannakakis.semijoin(self.rels[i], self.rels[i + 1], left_join_idx=1, right_join_idx=0)

    def reduce_top_down(self):
        # top-down: for i = 1 .. k-1, reduce R_{i+1} using R_i
        for i in range(len(self.rels) - 1):
            self.rels[i + 1] = SimpleYannakakis.semijoin(self.rels[i + 1], self.rels[i], left_join_idx=0, right_join_idx=1)

    def enumerate_join(self):
        # naive enumeration after reduction: perform left-deep expansion
        if not self.rels:
            return []
        result = [t for t in self.rels[0]]  # tuples of width 2 initially
        for i in range(1, len(self.rels)):
            nxt = self.rels[i]
            # build index on nxt by its first attribute
            index = defaultdict(list)
            for a,b in nxt:
                index[a].append(b)
            new_result = []
            for r in result:
                key = r[-1]
                if key in index:
                    for val in index[key]:
                        new_result.append(r + (val,))
            result = new_result
        return result

    def run(self):
        self.reduce_bottom_up()
        self.reduce_top_down()
        return self.enumerate_join()

# Left-deep hash join (line join)
def left_deep_hash_line_join(relations):
    if not relations:
        return []
    # start with R1 tuples
    result = [t for t in relations[0]]  # (A1,A2)
    for rel in relations[1:]:
        # build hash on rel's first attribute
        ht = defaultdict(list)
        for a,b in rel:
            ht[a].append(b)
        new_result = []
        for tup in result:
            key = tup[-1]
            for match in ht.get(key, []):
                new_result.append(tup + (match,))
        result = new_result
    return result

# Run both algorithms and time them
relations = [R1, R2, R3]

# Yannakakis
y = SimpleYannakakis(relations)
t0 = time.perf_counter()
y_result = y.run()
t1 = time.perf_counter()
y_time = t1 - t0

# Line join
t2 = time.perf_counter()
line_result = left_deep_hash_line_join(relations)
t3 = time.perf_counter()
line_time = t3 - t2

# Compare as multisets
same = Counter(y_result) == Counter(line_result)

print(R1)
print(R2)
print(R3)

# Pretty print summary similar to your example
print(f"Yannakakis Execution Time: {y_time} seconds")
print(f"Line Join Execution Time: {line_time} seconds")
print(f"Are results the same: {same}")
print(f"First 10 Results from Yannakakis: {y_result[:10]}")
print(f"First 10 Results from Line Join: {line_result[:10]}")


[(1, 7), (2, 72), (3, 28), (4, 99), (5, 83), (6, 1), (7, 1), (8, 72), (9, 48), (10, 52), (11, 6), (12, 9), (13, 99), (14, 72), (15, 7), (16, 62), (17, 28), (18, 1), (19, 1), (20, 62), (21, 48), (22, 7), (23, 11), (24, 62), (25, 23), (26, 28), (27, 9), (28, 83), (29, 53), (30, 6), (31, 49), (32, 6), (33, 53), (34, 68), (35, 30), (36, 99), (37, 6), (38, 28), (39, 6), (40, 52), (41, 83), (42, 53), (43, 13), (44, 53), (45, 72), (46, 30), (47, 7), (48, 48), (49, 9), (50, 48), (51, 28), (52, 99), (53, 7), (54, 73), (55, 68), (56, 49), (57, 30), (58, 73), (59, 9), (60, 83), (61, 28), (62, 23), (63, 49), (64, 28), (65, 49), (66, 53), (67, 23), (68, 72), (69, 9), (70, 7), (71, 28), (72, 53), (73, 28), (74, 28), (75, 53), (76, 23), (77, 99), (78, 52), (79, 30), (80, 99), (81, 48), (82, 73), (83, 53), (84, 72), (85, 99), (86, 53), (87, 13), (88, 11), (89, 1), (90, 1), (91, 1), (92, 1), (93, 23), (94, 6), (95, 53), (96, 23), (97, 49), (98, 99), (99, 72), (100, 62)]
[(53, 1), (13, 2), (1, 3), (23, 

In [30]:
# Problem 5
import random
import time
from collections import defaultdict, Counter

# Dataset creation (Problem 5)
random.seed(12345)

def make_R1():
    # 1000 tuples (i,5) for i=1..1000
    part1 = [(i, 5) for i in range(1, 1001)]
    # 1000 tuples (i,7) for i=1001..2000
    part2 = [(i, 7) for i in range(1001, 2001)]
    # add tuple (2001,2002)
    extra = [(2001, 2002)]
    R1 = part1 + part2 + extra
    random.shuffle(R1)
    return R1

def make_R2():
    # 1000 tuples (5,i) for i=1..1000
    part1 = [(5, i) for i in range(1, 1001)]
    # 1000 tuples (7,i) for i=1001..2000
    part2 = [(7, i) for i in range(1001, 2001)]
    # add tuple (2002,8)
    extra = [(2002, 8)]
    R2 = part1 + part2 + extra
    random.shuffle(R2)
    return R2

def make_R3():
    # 2000 random tuples (x,y) with x in [2002,3000], y in [1,3000]
    random_tuples = [(random.randint(2002, 3000), random.randint(1, 3000)) for _ in range(2000)]
    # add tuple (8,30)
    extra = [(8, 30)]
    R3 = random_tuples + extra
    random.shuffle(R3)
    return R3

R1 = make_R1()
R2 = make_R2()
R3 = make_R3()

# def save_csv(filename, rows, header=("col1","col2")):
#     with open(filename, "w", newline="") as f:
#         writer = csv.writer(f)
#         writer.writerow(header)
#         writer.writerows(rows)
#     print(f"Saved {filename} ({len(rows)} rows)")

# save_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/R1_problem5.csv", R1, header=("a", "b"))
# save_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/R2_problem5.csv", R2, header=("c", "d"))
# save_csv("/content/drive/MyDrive/QP-Hw-4-Dataset/R3_problem5.csv", R3, header=("e", "f"))

print("Sizes: R1 =", len(R1), "R2 =", len(R2), "R3 =", len(R3))

# Yannakakis-like algorithm (Problem 2)
class YannakakisLite:
    def __init__(self, relations):
        # copy relations to avoid mutating caller
        self.rels = [list(r) for r in relations]

    @staticmethod
    def semijoin_keep(left, right, left_idx=1, right_idx=0):
        """Keep tuples in left whose left[left_idx] appears among right[right_idx]."""
        keys = {t[right_idx] for t in right}
        return [t for t in left if t[left_idx] in keys]

    def bottom_up(self):
        # for i = k-1 down to 1, reduce R_i using R_{i+1}
        k = len(self.rels)
        for i in range(k - 2, -1, -1):
            self.rels[i] = YannakakisLite.semijoin_keep(self.rels[i], self.rels[i + 1], left_idx=1, right_idx=0)

    def top_down(self):
        # for i = 1 .. k-1, reduce R_{i+1} using R_i
        k = len(self.rels)
        for i in range(k - 1):
            self.rels[i + 1] = YannakakisLite.semijoin_keep(self.rels[i + 1], self.rels[i], left_idx=0, right_idx=1)

    def enumerate_left_deep(self):
        # left-deep enumeration after reduction
        if not self.rels:
            return []
        result = [t for t in self.rels[0]]  # start with R1 tuples (A1,A2)
        for rel in self.rels[1:]:
            # build index on rel's first attribute
            index = defaultdict(list)
            for a, b in rel:
                index[a].append(b)
            new_result = []
            for tup in result:
                key = tup[-1]
                if key in index:
                    for val in index[key]:
                        new_result.append(tup + (val,))
            result = new_result
        return result

    def run(self):
        self.bottom_up()
        self.top_down()
        return self.enumerate_left_deep()


# Left-deep hash join (Problem 3)
def left_deep_hash_join(relations):
    if not relations:
        return []
    result = [t for t in relations[0]]  # start with R1 tuples
    for rel in relations[1:]:
        # hash rel on its first attribute
        hash_index = defaultdict(list)
        for a, b in rel:
            hash_index[a].append(b)
        new_result = []
        for tup in result:
            key = tup[-1]
            for match in hash_index.get(key, []):
                new_result.append(tup + (match,))
        result = new_result
    return result

# Run both on Problem 5 dataset and time
relations = [R1, R2, R3]

# Yannakakis run
y_algo = YannakakisLite(relations)
t0 = time.perf_counter()
y_res = y_algo.run()
t1 = time.perf_counter()
y_time = t1 - t0

# Line join run
t2 = time.perf_counter()
line_res = left_deep_hash_join(relations)
t3 = time.perf_counter()
line_time = t3 - t2

same = Counter(y_res) == Counter(line_res)

# Print summary similar to your requested format
print(f"Yannakakis Execution Time: {y_time:.6f} seconds")
print(f"Line Join Execution Time: {line_time:.6f} seconds")
print(f"Are results the same: {same}")
print(f"First 10 Results from Yannakakis: {y_res[:10]}")
print(f"First 10 Results from Line Join: {line_res[:10]}")
print("Yannakakis output size:", len(y_res))
print("Line join output size:", len(line_res))
print(R1)
print(R2)
print(R3)


Sizes: R1 = 2001 R2 = 2001 R3 = 2001
Yannakakis Execution Time: 0.001242 seconds
Line Join Execution Time: 0.638476 seconds
Are results the same: True
First 10 Results from Yannakakis: [(991, 5, 8, 30), (871, 5, 8, 30), (398, 5, 8, 30), (981, 5, 8, 30), (175, 5, 8, 30), (558, 5, 8, 30), (958, 5, 8, 30), (537, 5, 8, 30), (234, 5, 8, 30), (484, 5, 8, 30)]
First 10 Results from Line Join: [(991, 5, 8, 30), (871, 5, 8, 30), (398, 5, 8, 30), (981, 5, 8, 30), (175, 5, 8, 30), (558, 5, 8, 30), (958, 5, 8, 30), (537, 5, 8, 30), (234, 5, 8, 30), (484, 5, 8, 30)]
Yannakakis output size: 1001
Line join output size: 1001
[(1399, 7), (1434, 7), (1003, 7), (1064, 7), (1739, 7), (1327, 7), (1862, 7), (1634, 7), (1750, 7), (991, 5), (871, 5), (1006, 7), (398, 5), (981, 5), (1413, 7), (1182, 7), (175, 5), (558, 5), (1599, 7), (1466, 7), (1806, 7), (958, 5), (1166, 7), (537, 5), (1467, 7), (1241, 7), (234, 5), (484, 5), (674, 5), (626, 5), (1294, 7), (976, 5), (1613, 7), (491, 5), (841, 5), (1988, 7), (

In [31]:
# Problem 7 - Query Processing
# q(A1,A2,A3,A4,A5,A6) :- R1(A1,A2), R2(A2,A3), R3(A1,A3), R4(A3,A4), R5(A4,A5), R6(A5,A6), R7(A4,A6)

import csv
import time
import os
from collections import defaultdict
DATA_DIR = "/content/drive/MyDrive/QP-Hw-4-Dataset/"


def load_csv(filename):
    full_path = os.path.join(DATA_DIR, filename)
    tuples = []
    with open(full_path, 'r') as f:
        rdr = csv.reader(f)
        next(rdr)  # skip header
        for row in rdr:
            if row:
                tuples.append((int(row[0]), int(row[1])))
    return tuples


def load_data():
    data = {}
    for i in range(1, 8):
        filename = f"R{i}.csv"   # ← Change name pattern if needed
        data[f"R{i}"] = load_csv(filename)
    return data


def make_indexes(data):
    idx = {}
    for i in range(1, 8):
        name = 'R' + str(i)
        idx[name + '_0'] = defaultdict(list)
        idx[name + '_1'] = defaultdict(list)
        for tup in data[name]:
            idx[name + '_0'][tup[0]].append(tup)
            idx[name + '_1'][tup[1]].append(tup)

    idx['proj_R2_0'] = set(t[0] for t in data['R2'])
    idx['proj_R4_0'] = set(t[0] for t in data['R4'])
    idx['proj_R5_0'] = set(t[0] for t in data['R5'])
    idx['proj_R7_0'] = set(t[0] for t in data['R7'])
    idx['proj_R6_0'] = set(t[0] for t in data['R6'])
    return idx


# GenericJoin - rho* = 3, O(N^3)
def genericjoin(data, idx):
    results = []
    proj_r2 = idx['proj_R2_0']
    proj_r4 = idx['proj_R4_0']
    proj_r5 = idx['proj_R5_0']
    proj_r7 = idx['proj_R7_0']
    proj_r6 = idx['proj_R6_0']

    vals_a1 = set(t[0] for t in data['R1']) & set(t[0] for t in data['R3'])

    for a1 in vals_a1:
        vals_a2 = set(t[1] for t in idx['R1_0'].get(a1, []))
        vals_a2 = vals_a2 & proj_r2

        for a2 in vals_a2:
            from_r2 = set(t[1] for t in idx['R2_0'].get(a2, []))
            from_r3 = set(t[1] for t in idx['R3_0'].get(a1, []))
            vals_a3 = from_r2 & from_r3 & proj_r4

            for a3 in vals_a3:
                from_r4 = set(t[1] for t in idx['R4_0'].get(a3, []))
                vals_a4 = from_r4 & proj_r5 & proj_r7

                for a4 in vals_a4:
                    from_r5 = set(t[1] for t in idx['R5_0'].get(a4, []))
                    vals_a5 = from_r5 & proj_r6

                    for a5 in vals_a5:
                        from_r6 = set(t[1] for t in idx['R6_0'].get(a5, []))
                        from_r7 = set(t[1] for t in idx['R7_0'].get(a4, []))
                        vals_a6 = from_r6 & from_r7

                        for a6 in vals_a6:
                            results.append((a1, a2, a3, a4, a5, a6))
    return results


# GHW - ghw = 2, O(N^2)
def ghw(data):
    r2_hash = defaultdict(list)
    for t in data['R2']:
        r2_hash[t[0]].append(t)

    bag1_temp = []
    for t1 in data['R1']:
        for t2 in r2_hash.get(t1[1], []):
            bag1_temp.append((t1[0], t1[1], t2[1]))

    r3_pairs = set((t[0], t[1]) for t in data['R3'])
    bag1 = [t for t in bag1_temp if (t[0], t[2]) in r3_pairs]

    bag2 = list(data['R4'])

    r6_hash = defaultdict(list)
    for t in data['R6']:
        r6_hash[t[0]].append(t)

    bag3_temp = []
    for t5 in data['R5']:
        for t6 in r6_hash.get(t5[1], []):
            bag3_temp.append((t5[0], t5[1], t6[1]))

    r7_pairs = set((t[0], t[1]) for t in data['R7'])
    bag3 = [t for t in bag3_temp if (t[0], t[2]) in r7_pairs]

    # yannakakis
    bag2_a3 = set(t[0] for t in bag2)
    bag2_a4 = set(t[1] for t in bag2)
    bag1 = [t for t in bag1 if t[2] in bag2_a3]
    bag3 = [t for t in bag3 if t[0] in bag2_a4]

    bag1_a3 = set(t[2] for t in bag1)
    bag3_a4 = set(t[0] for t in bag3)
    bag2 = [t for t in bag2 if t[0] in bag1_a3 and t[1] in bag3_a4]

    bag2_by_a3 = defaultdict(list)
    for t in bag2:
        bag2_by_a3[t[0]].append(t)

    temp_result = []
    for t in bag1:
        for s in bag2_by_a3.get(t[2], []):
            temp_result.append((t[0], t[1], t[2], s[1]))

    bag3_by_a4 = defaultdict(list)
    for t in bag3:
        bag3_by_a4[t[0]].append(t)

    final = []
    for t in temp_result:
        for s in bag3_by_a4.get(t[3], []):
            final.append((t[0], t[1], t[2], t[3], s[1], s[2]))
    return final


def triangle_gj(r1, r2, r3):
    r1_by_x = defaultdict(list)
    for t in r1:
        r1_by_x[t[0]].append(t)

    r2_by_y = defaultdict(list)
    for t in r2:
        r2_by_y[t[0]].append(t)

    r3_by_x = defaultdict(list)
    for t in r3:
        r3_by_x[t[0]].append(t)

    x_vals = set(r1_by_x.keys()) & set(r3_by_x.keys())

    result = []
    for x in x_vals:
        y_vals = set(t[1] for t in r1_by_x[x])
        for y in y_vals:
            z_from_r2 = set(t[1] for t in r2_by_y.get(y, []))
            z_from_r3 = set(t[1] for t in r3_by_x[x])
            z_vals = z_from_r2 & z_from_r3
            for z in z_vals:
                result.append((x, y, z))
    return result


# FHW - fhw = 1.5, O(N^1.5)
def fhw(data):
    bag1 = triangle_gj(data['R1'], data['R2'], data['R3'])
    bag2 = list(data['R4'])
    bag3 = triangle_gj(data['R5'], data['R6'], data['R7'])

    bag2_a3 = set(t[0] for t in bag2)
    bag2_a4 = set(t[1] for t in bag2)
    bag1 = [t for t in bag1 if t[2] in bag2_a3]
    bag3 = [t for t in bag3 if t[0] in bag2_a4]

    bag1_a3 = set(t[2] for t in bag1)
    bag3_a4 = set(t[0] for t in bag3)
    bag2 = [t for t in bag2 if t[0] in bag1_a3 and t[1] in bag3_a4]

    bag2_hash = defaultdict(list)
    for t in bag2:
        bag2_hash[t[0]].append(t)

    temp = []
    for t in bag1:
        for s in bag2_hash.get(t[2], []):
            temp.append((t[0], t[1], t[2], s[1]))

    bag3_hash = defaultdict(list)
    for t in bag3:
        bag3_hash[t[0]].append(t)

    final = []
    for t in temp:
        for s in bag3_hash.get(t[3], []):
            final.append((t[0], t[1], t[2], t[3], s[1], s[2]))
    return final


# run
print("Loading data...")
data = load_data()
idx = make_indexes(data)

for i in range(1, 8):
    print("  R{}: {} tuples".format(i, len(data['R' + str(i)])))

n = max(len(data['R' + str(i)]) for i in range(1, 8))
print("N = {}".format(n))

print("\nGenericJoin (rho* = 3, O(N^3))")
t1 = time.time()
res1 = genericjoin(data, idx)
t1 = time.time() - t1
print("Output: {} tuples, Time: {:.4f} sec".format(len(res1), t1))

print("\nGHW (ghw = 2, O(N^2))")
t2 = time.time()
res2 = ghw(data)
t2 = time.time() - t2
print("Output: {} tuples, Time: {:.4f} sec".format(len(res2), t2))

print("\nFHW (fhw = 1.5, O(N^1.5))")
t3 = time.time()
res3 = fhw(data)
t3 = time.time() - t3
print("Output: {} tuples, Time: {:.4f} sec".format(len(res3), t3))

# verify
s1 = set(res1)
s2 = set(res2)
s3 = set(res3)
if s1 == s2 == s3:
    print("\nAll algorithms give same result: {} tuples".format(len(s1)))
else:
    print("\nERROR - results don't match!")

print("\n{:<15} {:<10} {:<12} {:<10}".format("Algorithm", "Width", "Complexity", "Time"))
print("{:<15} {:<10} {:<12} {:.4f}".format("GenericJoin", "rho*=3", "O(N^3)", t1))
print("{:<15} {:<10} {:<12} {:.4f}".format("GHW", "ghw=2", "O(N^2)", t2))
print("{:<15} {:<10} {:<12} {:.4f}".format("FHW", "fhw=1.5", "O(N^1.5)", t3))

print("\nTheoretical: N^3={}, N^2={}, N^1.5={:.0f}".format(n**3, n**2, n**1.5))

fastest = min(t1, t2, t3)
print("\nRelative times: GJ={:.2f}x, GHW={:.2f}x, FHW={:.2f}x".format(
    t1/fastest, t2/fastest, t3/fastest))

Loading data...
  R1: 100 tuples
  R2: 100 tuples
  R3: 100 tuples
  R4: 100 tuples
  R5: 100 tuples
  R6: 100 tuples
  R7: 100 tuples
N = 100

GenericJoin (rho* = 3, O(N^3))
Output: 1000000 tuples, Time: 0.7561 sec

GHW (ghw = 2, O(N^2))
Output: 1000000 tuples, Time: 0.3550 sec

FHW (fhw = 1.5, O(N^1.5))
Output: 1000000 tuples, Time: 0.3417 sec

All algorithms give same result: 1000000 tuples

Algorithm       Width      Complexity   Time      
GenericJoin     rho*=3     O(N^3)       0.7561
GHW             ghw=2      O(N^2)       0.3550
FHW             fhw=1.5    O(N^1.5)     0.3417

Theoretical: N^3=1000000, N^2=10000, N^1.5=1000

Relative times: GJ=2.21x, GHW=1.04x, FHW=1.00x
